In [ ]:
import requests
from tqdm import tqdm

# URL of the file to be downloaded
url = "https://public.boxcloud.com/d/1/b1!DF-5pZffA7zR1_FcUVCjx4PJaM3r8Lx6kBOjMXR9DG-geexQh9pfYrZNUzBLVe5vu620bSdhYCFpRhlwpF8DRlkV1VXDUIytUAH0lEvnJsN2Gzu4IVGKYqvRCwEQw02lbOAxCKDHJ7tKVn-C2OQSZs6Cr2kCOHM5TLQxIfWzrJzTU4dxHicqi-tDUeMF4xo2WKxIH8aNtVmx5TIMa_9tWaOb3npDRkUvk7MGCuk6CFsQSTNigqvmC2gSRtBnrVtdnCDPZF_yNBBLCpyOVFAEeR18pnqSdCw0HS4nkDHeA9yZ1RF4t6eYRM4xK8DrWAWMdDmT2_qdw4g4wi8WSjXeCCUmE3kOaUcmd3pXgS46qtb8lZdmw4PTriZUt9szpaRe5AgxO7F_Up0b8mbNZWDJ8kF7G9gHR6vToa28kWq_TwF7nOEtbdzPHZeL-vAk73D2cCJTmJB4einKCrsGLe4R-MOzKcWj7wu2Fjt1IvS6aqVRVy1uvwIRiWrUyhHDxSxsFGezirHsGKbBQej5Cytn40BaOlmFHMvK3S9vfpL8m2XjqmBW_q6sw4HnNYMBKTb9DzKrQb2aj3apEyuANeh-VSImgX09kiIWM70Q-QpUAI794N8bnJ3iykg6kjWro-3EZb44wOjuvMNGUp-HOzAbwY2P_Gk2y-Opo3UkXwoJV66n0LyMVgt8tLI0Dm7Q_VEsyw9u-b_mn3-uxblZR4fllyfCur7Ew8R2jiIZcry-o5iE2M1lIL7zlDy3-yCMhN75VXMI6EVvtuIwaHmLMubvu2N62nGVJ2n039LVAkByCoGzRBNIOXVgBy-aY49xnScn0F_fzyjQIK7VdsObJnO9Tg5bjurfRQ_ZQNx-g4CPG-vne2FkuI1ft7bLEcIMmVk0zn67VHwmcQ4rcBFHW_d_Vw2OfozrS0QjpcOIX1oC7h_MZIHb2SMzLKu3VyhqroZkYSz-Eebl3ieQuF8NO9d4kRJin_OBxki1_7IFSa239hcMoMk3x3hmHmsfeOb3msEtCdcXvcQOdt-inyjuNlvd_uhqs-p2LWcW32GBpAfYpnP26wMT-DHo1lX7k4R4BEsxWIelC-_-GnGpx7R7wsxvHmamyjjKeNfv_RvUeBYGqC67q64HAZY3ggQyyuzCanzXukzevdO5CkmxB61cNAE4OfKYcuPpITWE9XStCjA_hQ7xsnSz6tG0v69Ul9-qWxrmlChf9aubNe6HyBifPh7l7udwRnaVumiTMN0HaM232VdT-t5HoetaqSCnKIj46ql4iEZOqCuA/download"

# Send a GET request to the URL to get the file size
response = requests.head(url, allow_redirects=True)
file_size = int(response.headers.get('content-length', 0))

# Stream the download
response = requests.get(url, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # Define the filename to save the downloaded file
    filename = "atlasv2.tar.gz"

    # Open a local file in binary write mode
    with open(filename, 'wb') as file, tqdm(
        desc=filename,
        total=file_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
            bar.update(len(chunk))

    print(f"File downloaded successfully and saved as {filename}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

In [ ]:
!tar -xzvf atlasv2.tar.gz -C atlas_dataset/

In [ ]:
import pandas as pd

# Define the path to the JSONL file
logs_path = "atlas_dataset/atlasv2/data/benign/h1/cbc-edr/edr-h1-benign.jsonl"

# Load the JSONL file into a pandas DataFrame
df = pd.read_json(logs_path, lines=True)

# Display the first few rows of the DataFrame
print(df.head())

In [ ]:
from pprint import pprint

logs_path = "atlas_dataset/atlasv2/data/benign/h1/cbc-edr/edr-h1-benign.jsonl"

# Initialize an empty list to store the lines
first_few_lines = []

# Specify the number of lines you want to read
num_lines_to_read = 50

# Open the file and read the specified number of lines
with open(logs_path, 'r') as file:
    for _ in range(num_lines_to_read):
        line = file.readline()
        if not line:
            break  # Stop if the file has fewer lines than num_lines_to_read
        first_few_lines.append(line.strip())

# Print the first few lines
for line in first_few_lines:
    pprint(line)
    print("\n")

In [ ]:
set(df['type'])

In [ ]:
import json

# Define the path to the log file
logs_path = "atlas_dataset/atlasv2/data/benign/h1/cbc-edr/edr-h1-benign.jsonl"

# Initialize a dictionary to store one example of each event type
event_types = {
    'endpoint.event.crossproc': None,
    'endpoint.event.filemod': None,
    'endpoint.event.moduleload': None,
    'endpoint.event.netconn': None,
    'endpoint.event.procend': None,
    'endpoint.event.procstart': None,
    'endpoint.event.regmod': None,
    'endpoint.event.scriptload': None,
}

# Read the log file and find one event for each type
try:
    with open(logs_path, 'r') as file:
        for line in file:
            event = json.loads(line)
            event_type = event.get('type')
            if event_type in event_types and event_types[event_type] is None:
                event_types[event_type] = event
            # Check if we have found all event types
            if all(event_types.values()):
                break

    # Print the events
    for event_type, event in event_types.items():
        print(f"Event Type: {event_type}\nEvent: {json.dumps(event, indent=2)}\n")

except FileNotFoundError:
    print(f"File not found: {logs_path}")
except Exception as e:
    print(f"An error occurred: {e}")

In [2]:
import pandas as pd
import json
import os
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Ensure CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the log data from a JSON Lines file
file_path = "atlas_dataset/atlasv2/data/benign/h1/cbc-edr/edr-h1-benign.jsonl"  # Replace with your actual file path

log_data = []
with open(file_path, 'r') as file:
    for line in file:
        log_data.append(line)

# Use a model
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# Define a function to analyze each event using a Hugging Face model
def analyze_event(event_details):
    prompt = f"Analyze the system logs and give insights about what is contained in the event {event_details}"
    prompt = "what is sun?"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    inputs = inputs.to(device)
    with torch.no_grad():
        outputs = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"],
                                 max_new_tokens=50, do_sample=True, top_k=50, top_p=0.95)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()

# Example of analyzing the first event in the log data
print(analyze_event(log_data[0]))


Using device: cuda


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


what is sun? What is day? Sun is the source of energy. Day is the way in which energy is used.

"Sun rises tomorrow. The day is the way in which that energy is used. How do people know which way the sun is going


In [ ]:
import pandas as pd
import json
import os
import openai

# Set up the OpenAI API key from an environment variable
openai.api_key = "sk-proj-hJElIpLWIfFxcFsECXDeT3BlbkFJBQSjlqsoMgQ833cW0hYY"

# Load the log data from a JSON Lines file
file_path = "atlas_dataset/atlasv2/data/benign/h1/cbc-edr/edr-h1-benign.jsonl"  # Replace with your actual file path

log_data = []
with open(file_path, 'r') as file:
    for line in file:
        log_data.append(line)  # Assuming JSON format, use json.loads

# Define a function to analyze each event using the OpenAI API
def analyze_event(event_details):
    prompt = f"Analyze the system logs and give insights about what is contained in the event: {event_details}"
    response = openai.Completion.create(
      engine="text-davinci-003",  # Choose the appropriate model version
      prompt=prompt,
      max_tokens=150,
      temperature=0.5
    )
    return response.choices[0].text.strip()

# Example of analyzing the first event in the log data
if log_data:  # Check if log_data is not empty
    print(analyze_event(log_data[0]))
else:
    print("No data found in the log file.")

In [ ]:
import openai

# Set up the OpenAI API key from an environment variable
openai.api_key = "sk-proj-hJElIpLWIfFxcFsECXDeT3BlbkFJBQSjlqsoMgQ833cW0hYY"  

# Load the log data from a JSON Lines file
file_path = "atlas_dataset/atlasv2/data/benign/h1/cbc-edr/edr-h1-benign.jsonl"  # Replace with your actual file path

log_data = []
with open(file_path, 'r') as file:
    for line in file:
        log_data.append(line)  # Assuming JSON format, use json.loads

# Define a function to analyze each event using the OpenAI API
def analyze_event(event_details):
    prompt = f"Analyze the system logs and give insights about what is contained in the event: {event_details}"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Updated to use a newer model
        messages=[{"role": "system", "content": prompt}],
        max_tokens=150
    )
    return response['choices'][0]['message']['content'].strip()

# Example usage
if log_data:
    event_json = json.loads(log_data[0])  # Make sure to parse the JSON line
    print(analyze_event(event_json))
else:
    print("No data found in the log file.")


In [1]:
import pandas as pd
import json
import os
import openai

# Set up the OpenAI API key from an environment variable
openai.api_key = "sk-proj-hJElIpLWIfFxcFsECXDeT3BlbkFJBQSjlqsoMgQ833cW0hYY"

# Load the log data from a JSON Lines file
file_path = "atlas_dataset/atlasv2/data/benign/h1/cbc-edr/edr-h1-benign.jsonl"  # Replace with your actual file path

log_data = []
with open(file_path, 'r') as file:
    for line in file:
        log_data.append(json.loads(line))  # Assuming JSON format, use json.loads

# Define a function to analyze each event using the OpenAI API
def analyze_event(event_details):
    prompt = f"Analyze the system logs and give insights about what is contained in the event: {event_details}"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Choose the appropriate model version
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.5
    )
    return response['choices'][0]['message']['content'].strip()

# Example of analyzing the first event in the log data
if log_data:  # Check if log_data is not empty
    print(analyze_event(log_data[0]))
else:
    print("No data found in the log file.")


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.